## Chap07_물류 네트워크 최적 설계

**상황**
* '물류 네트워크'가 포함된 경영 현황을 개선
    
→ 개별 최적화뿐만 아니라, **물류 네트워크 전체의 최적화**가 필요 

**7장 내용**
* 네트워크 가시화 기술로 계산 결과의 타당성 확인 

**고객의 소리**

창고에서 공장까지의 운송 비용을 조정하는 것만으로도 최적화 가능성이 보였습니다. 계속해서 분석을 부탁드립니다. 이왕이면 우리 회사의 제조에서 물류까지의 전체 흐름 중 어디에 비용 개선 가능성이 있는지 분석해 주셨으면 합니다.

**전제조건**

* 최종적으로 제품을 판매하는 대리점(P, Q)
* 대리점에 판매되는 상품군(제품 A, B)에는 일정 수요가 예측되어 있어, 이 수요량을 근거로 공장(공장 X, Y)에서의 생산량을 결정
* 각각 제품을 어느 공장의 어느 생산라인(레인 0, 1)에서 제조할지에 대해서는 각 공장에서 대리점으로까지의 운송비, 재고 비용 등을 고려해서 결정 

### 테크닉 61 : 운송 최적화 문제

* 사용할 라이브러리 
    
    - **pulp** : 선형 프로그래밍 문제를 모델링하고 최적화 모델을 작성
    - **ortoolpy** : 목적함수를 생성해서 최적화 문제를 푸는 역할

**PuLP 라이브러리**
* LpVariable :  선형 프로그래밍 변수 생성
* lpSum : 변수의 합
* value : 변수의 값에 접근

**ortoolpy 라이브러리**
* model_min : 최소 비용 모델 생성하는데 사용
* addvars : 변수를 모델에 추가하는 데 사용

In [1]:
import numpy as np
import pandas as pd
from itertools import product # 여러 집합의 카르테시안 곱을 생성
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars

# 데이터 불러오기 
df_tc = pd.read_csv('trans_cost.csv', index_col = "공장") #  공장 간의 운송 비용 정보
df_demand = pd.read_csv('demand.csv') # 수요 정보 
df_supply = pd.read_csv('supply.csv') # 공급 정보 
display(df_tc, df_demand, df_supply)

# 초기 설정 
np.random.seed(1) # 난수 발생 제어
nw = len(df_tc.index) # 'df_tc' 의 행 수, 공장의 수
nf = len(df_tc.columns) # 'df_tc'의 열 수, 공급처의 수 
pr = list(product(range(nw), range(nf))) #  모든 공장 및 공급처의 조합


,F1,F2,F3,F4
공장,,,,
W1,10,10,11,27
W2,18,21,12,14
W3,15,12,14,12


,F1,F2,F3,F4
0,28,29,31,25


,W1,W2,W3
0,35,41,42


**운송 최적화 문제**

* 최적화 문제는 목적함수와 제약 조건이 정의돼야 풀 수 있다. 

In [2]:
# 수리 모델 작성
# '최소화를 실행하는' 모델로 m1 정의 → 정의하는 목적함수를 제약 조건 하에서 '최소화'할 수 있다.
m1 = model_min() # '최소화를 실행하는' 모델로 m1 정의 
v1 = {(i, j) : LpVariable('v%d_%d'%(i, j), lowBound = 0) for i, j in pr} # 음수가 아닌 운송량 변수

m1 += lpSum(df_tc.iloc[i][j] * v1[i, j] for i, j in pr)
for i in range(nw) : # 공장의 수만큼 반복
    m1 += lpSum(v1[i, j] for j in range(nf)) <= df_supply.iloc[0][i]
for j in range(nf) : # 공급처의 수만큼 반복
    m1 += lpSum(v1[i, j] for i in range(nw)) >= df_demand.iloc[0][j]
m1.solve()

# 총 운송 비용 계산 
df_tr_sol = df_tc.copy()
total_cost = 0
for k, x in v1.items() :
    i, j = k[0], k[1]
    df_tr_sol.iloc[i][j] = value(x)
    total_cost += df_tc.iloc[i][j] * value(x)
    
print(df_tr_sol)
print("총 운송 비용 :" + str(total_cost))



    F1  F2  F3  F4
공장                
W1  28   7   0   0
W2   0   0  31   5
W3   0  22   0  20
총 운송 비용 :1296.0
